In [1]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

% matplotlib inline

In [2]:
num_image = 50000

In [7]:
def add_images_from_files_to_list(root, files, list, next_index, data_cb=None):
    if files and next_index < num_image:
        def identity(v):
            return v

        if data_cb is None:
            data_cb = identity

        for index, file in enumerate(files):
            image = cv2.imread(os.path.join(root, file))

            if image is not None and next_index < num_image:
                list[next_index] = data_cb(np.average(image, axis=2).reshape((96, 96, 1)))
                next_index += 1
                # list.append(data_cb(image[:, :, 0]))
    return next_index

In [8]:
images_train = np.empty((num_image, 96, 96, 1))
i = 0

In [9]:
for root, sub_directories, files in tqdm(os.walk("./danbooru-faces")):
    if i < num_image:
        i = add_images_from_files_to_list(root, files, images_train, i)

# plt.subplot(96, 96)
# plt.imshow(images_train[20], interpolation='nearest', cmap='gray_r')
# plt.axis('off')
# plt.tight_layout()
# plt.savefig('./collected_data/gan_generated_image_epoch_%d.png' % 999)
# plt.clf()

0it [00:00, ?it/s]

AttributeError: 'list' object has no attribute 'close'

In [6]:
# images_train = np.array(images_train)
# images_train = images_train.reshape((*images_train.shape, 1))
# images_train = images_train[:, :, :, :1]

In [7]:
print(images_train.shape)

(50000, 96, 96, 1)

In [8]:
from keras.layers import BatchNormalization, Dropout
from keras.layers import Conv2D, MaxPool2D, Flatten
from keras.layers import Conv2DTranspose, UpSampling2D
from keras.layers import Dense, Activation, Reshape
from keras.models import Input, Model
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [9]:
dropout = 0.4
depth = 256
dim = 8
input_dim = 128
batch_size = 50
epochs = 200

In [10]:
def keras_generator():
    g = Sequential()
    # In: 100
    # Out: dim x dim x depth
    g.add(Dense(dim * dim * depth, input_dim=input_dim))
    # g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    g.add(Reshape((dim, dim, depth)))
    g.add(Dropout(dropout))
    # In: dim x dim x depth
    # Out: 2*dim x 2*dim x depth/2
    g.add(UpSampling2D())
    g.add(Conv2DTranspose(int(depth / 2), 5, padding='same'))
    # g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))

    g.add(UpSampling2D())
    g.add(Conv2DTranspose(int(depth / 4), 5, padding='same'))
    # g.add(BatchNormalization(momentum=0.9))
    g.add(Activation('relu'))
    # In: dim x dim x depth
    g.add(Conv2DTranspose(1, 5, padding='same'))
    g.add(Activation('relu'))

    g.add(Reshape((4*dim * 4*dim,)))

    # # Out: 96 x 96 x 1 grayscale image
    g.add(Dense(96 * 96))
    g.add(Reshape((96, 96, 1)))
    g.add(Activation('tanh'))
    g.summary()
    g.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

    return g

In [11]:
def keras_discriminator():
    """
        Translation of tutorial into Keras.
    """
    inputs = Input(shape=images_train.shape[1:])
    layer = Conv2D(filters=32, kernel_size=(5, 5), activation="relu")(inputs)
    layer = MaxPool2D()(layer)
    layer = Conv2D(filters=64, kernel_size=(5, 5), activation="relu")(layer)
    layer = MaxPool2D()(layer)
    layer = Conv2D(filters=128, kernel_size=(5, 5), activation="relu")(layer)
    layer = MaxPool2D()(layer)

    layer = Flatten()(layer)
    layer = Dense(units=512, activation="relu")(layer)
    # layer = Dense(units=1024, activation="relu")(layer)
    layer = Dense(units=128, activation="relu")(layer)
    outputs = Dense(units=1, activation="sigmoid")(layer)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    model.summary()

    return model

In [12]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

In [13]:
# generator1 = keras_generator()
# generator2 = keras_generator()
# generator3 = keras_generator()

generator = keras_generator()

ganInput = Input((input_dim,))

# x1 = generator1(ganInput)
# x2 = generator2(ganInput)
# x3 = generator3(ganInput)

x = generator(ganInput)

_________________________________________________________________

Layer (type)                 Output Shape              Param #   

dense_1 (Dense)              (None, 16384)             2113536   

_________________________________________________________________

activation_1 (Activation)    (None, 16384)             0         

...

activation_5 (Activation)    (None, 96, 96, 1)         0         

Total params: 12,585,729

Trainable params: 12,585,729

Non-trainable params: 0

_________________________________________________________________

In [14]:
discriminator = keras_discriminator()
# d_input = keras.layers.concatenate([x1, x2, x3], axis=-1)
# d_input = Reshape((96, 96, 3))(d_input)
# generator = Model(inputs=ganInput, outputs=d_input)
# ganOutput = discriminator(d_input)
ganOutput = discriminator(x)

_________________________________________________________________

Layer (type)                 Output Shape              Param #   

input_2 (InputLayer)         (None, 96, 96, 1)         0         

_________________________________________________________________

conv2d_1 (Conv2D)            (None, 92, 92, 32)        832       

...

dense_5 (Dense)              (None, 1)                 129       

Total params: 4,517,633

Trainable params: 4,517,633

Non-trainable params: 0

_________________________________________________________________

In [15]:
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [16]:
dLosses = []
gLosses = []

In [17]:
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('./collected_data/gan_loss_epoch_%d.png' % epoch)
    plt.clf()

In [18]:
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, input_dim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 96, 96)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('./collected_data/gan_generated_image_epoch_%d.png' % epoch)
    plt.clf()

In [19]:
# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    generator.save('./collected_models/gan_generator1_epoch_%d.h5' % epoch)
    # generator1.save('./collected_models/gan_generator1_epoch_%d.h5' % epoch)
    # generator2.save('./collected_models/gan_generator2_epoch_%d.h5' % epoch)
    # generator3.save('./collected_models/gan_generator3_epoch_%d.h5' % epoch)
    discriminator.save('./collected_models/gan_discriminator_epoch_%d.h5' % epoch)

In [20]:
def train(epochs=1, batchSize=128):
    batchCount = int(images_train.shape[0] / batchSize)
    print('Epochs:', epochs)
    print('Batch size:', batchSize)
    print('Batches per epoch:', batchCount)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        dloss = 0
        gloss = 0
        for b in range(batchCount):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, input_dim])
            imageBatch = images_train[np.random.randint(0, images_train.shape[0], size=batchSize)]

            # Generate fake MNIST images
            generatedImages = generator.predict(noise)
            # print(np.shape(imageBatch), np.shape(generatedImages))
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            discriminator.trainable = True
            (dloss, dAcc) = discriminator.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, input_dim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

            # print('-'*5, 'Batch %d/%d done' % ((b + 1), batchCount))

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        plotGeneratedImages(e)
        if e == 1 or e % 20 == 0:
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)

In [21]:
train(epochs, batch_size)

Epochs:

200

Batch size:

50

Batches per epoch:

...

---------------

---------------

Epoch 10

---------------

KeyboardInterrupt: 